In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
cursor_object = shelter.readAll()

df = pd.DataFrame.from_records(cursor_object)



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H2('Joseph Collett'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),    
    html.Hr(),
    html.Div(        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
        id='filter-type',
        # created the labels and keys based on the Grazioso requirements
        options=[
            {'label': 'Water Rescue', 'value': 'water-rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain-wilderness-rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster-rescue-individual-tracking'},
            {'label': 'Reset and Undo Filters', 'value': 'reset'}
        ],
        labelStyle={'display': 'inline-block'}
        )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        #If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_option):
### FIX ME Add code to filter interactive data table with MongoDB queries  
        username = "aacuser"
        password = "aacuser"
        shelter = AnimalShelter(username, password)
        
        if filter_option == 'water-rescue':
            cursor_object = shelter.readFilterWR()
            df = pd.DataFrame.from_records(cursor_object)
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
            data=df.to_dict('records')    
            return (data,columns)
        
        elif filter_option == 'mountain-wilderness-rescue':
            cursor_object = shelter.readFilterMWR()
            df = pd.DataFrame.from_records(cursor_object)
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
            data=df.to_dict('records')    
            return (data,columns)
        
        elif filter_option == 'disaster-rescue-individual-tracking':
            cursor_object = shelter.readFilterDIT()
            df = pd.DataFrame.from_records(cursor_object)
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
            data=df.to_dict('records')    
            return (data,columns)
        
        elif filter_option == 'reset':
            print(filter_option)
            cursor_object = shelter.readAll()
            df = pd.DataFrame.from_records(cursor_object)
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
            data=df.to_dict('records')    
            return (data,columns)
        
        else:
            raise dash.exceptions.PreventUpdate

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]
    dff = pd.DataFrame.from_dict(viewData)
    new_df = dff.breed.value_counts()
    new_df.name = "breed"
    new_df = new_df.to_frame()
    new_df.reset_index(inplace=True)

    # return updated graph data
    return [
    dcc.Graph(
    figure = px.pie(new_df, values="breed", names="index")
    )
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app